In [1]:
import pandas as pd
import datetime
import os


# load normal data

In [2]:
dfs = [pd.read_parquet(f"./DNS-Tunnel-Datasets/normal/normal/{f}") for f in 
              os.listdir('./DNS-Tunnel-Datasets/normal/normal/') if f.endswith('.pq')]

cols_unwanted = ['payload.ack', 'payload.dataofs', 'payload.flags', 'payload.reserved', 'payload.seq', 'payload.urgptr', 'payload.window']
dfs = [df[[c for c in df.columns if c not in cols_unwanted]] for df in dfs]

df_normal = pd.concat(dfs)

# for f in [f for f in os.listdir('./DNS-Tunnel-Datasets/tunnel/') if f.endswith('.pq')]


/var/folders/vt/4l69jtfd3m5blk3vs5plrg_40000gp/T/ipykernel_10797/3858785897.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_normal = pd.concat(dfs)


In [3]:
# sanity check

df_normal['qname'].value_counts()

qname
img1.wsimg.com.                      764
www.google.com.                      726
performance.radar.cloudflare.com.    630
d38psrni17bvxu.cloudfront.net.       574
cdn.jsdelivr.net.                    536
                                    ... 
pro-technology.com.                    2
outplay-helios.com.                    2
pokerdomdream.ru.                      2
pokerdom-3947.top.                     2
plmlatina.net.                         2
Name: count, Length: 974410, dtype: int64

# load tunneling data

In [4]:
dfs = [pd.read_parquet(f"./DNS-Tunnel-Datasets/tunnel/{f}") for f in 
              os.listdir('./DNS-Tunnel-Datasets/tunnel/') if f.endswith('.pq')]

cols_unwanted = ['payload.ack', 'payload.dataofs', 'payload.flags', 'payload.reserved', 'payload.seq', 'payload.urgptr', 'payload.window']
dfs = [df[[c for c in df.columns if c not in cols_unwanted]] for df in dfs]

df_tunneling = pd.concat(dfs)

# for f in [f for f in os.listdir('./DNS-Tunnel-Datasets/tunnel/') if f.endswith('.pq')]


In [5]:
# quick sanity check
df_tunneling['qname']

0                               vaaaakaqoou.www.ggy666.tk.
1                               vaaaakaqoou.www.ggy666.tk.
2          lahxxmg4jl15duwvgubes0sidhbeq23q.www.ggy666.tk.
3          lahxxmg4jl15duwvgubes0sidhbeq23q.www.ggy666.tk.
4                                    yrbdtx.www.ggy666.tk.
                               ...                        
36692                                r13311.www.ggy666.tk.
36693                                r13310.www.ggy666.tk.
36694                                r13312.www.ggy666.tk.
36695    dIUIAAWBWWJAAAQAGDCFMBKBVAHAKQNICQ64AAFXGWPC76...
36696                                r13313.www.ggy666.tk.
Name: qname, Length: 375810, dtype: object

# join data, add label column

In [6]:
df_normal['label'] = 'normal'
df_tunneling['label'] = 'tunneling'

df_all = pd.concat([df_normal, df_tunneling])

/var/folders/vt/4l69jtfd3m5blk3vs5plrg_40000gp/T/ipykernel_10797/3589576660.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_normal, df_tunneling])


In [7]:
df_all['label'].value_counts()

label
normal       2012494
tunneling     375810
Name: count, dtype: int64

# start EDA

In [8]:
df_all.head()

,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,...,unicastresponse,qclass,tc,fl,plen,nh,hlim,payload.payload.ns,label,payload.payload.ar
0,4,5.0,0.0,63.0,31242.0,0.0,0.0,128.0,17.0,46867.0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,normal,NaN
1,4,5.0,0.0,79.0,52057.0,2.0,0.0,64.0,17.0,26036.0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,normal,NaN
2,4,5.0,0.0,55.0,31243.0,0.0,0.0,128.0,17.0,46874.0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,normal,NaN
3,4,5.0,0.0,71.0,52079.0,2.0,0.0,64.0,17.0,26022.0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,normal,NaN
4,4,5.0,0.0,63.0,31244.0,0.0,0.0,128.0,17.0,46865.0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,normal,NaN


In [9]:
# see if any columns are null

for c in df_all.columns:
    if df_all[c].isnull().sum() / df_all.shape[0] > 0.99:
        print(c)

payload.payload.length
tc
fl
plen
nh
hlim
payload.payload.ns


In [11]:
df_all.describe()

,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,...,payload.payload.nscount,payload.payload.arcount,qtype,unicastresponse,qclass,tc,fl,plen,nh,hlim
count,2.388304e+06,2388257.0,2388257.0,2.388257e+06,2.388257e+06,2.388257e+06,2388257.0,2.388257e+06,2.388257e+06,2.388257e+06,...,2.388304e+06,2.388304e+06,2.388304e+06,2388304.0,2388304.0,47.0,4.700000e+01,47.000000,47.0,47.0
mean,4.000039e+00,5.0,0.0,1.083597e+02,3.177891e+04,1.002148e+00,0.0,9.593132e+01,1.699996e+01,3.255354e+04,...,5.024486e-06,3.321185e-02,3.456407e+02,0.0,1.0,0.0,5.652976e+05,120.702128,17.0,64.0
std,8.872172e-03,0.0,0.0,1.434146e+02,1.933445e+04,9.999979e-01,0.0,3.199994e+01,2.013244e-02,1.860961e+04,...,2.241531e-03,1.791894e-01,4.727012e+03,0.0,0.0,0.0,2.616050e+05,87.178955,0.0,0.0
min,4.000000e+00,5.0,0.0,5.000000e+01,0.000000e+00,0.000000e+00,0.0,5.400000e+01,6.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,0.0,1.0,0.0,3.391900e+04,40.000000,17.0,64.0
25%,4.000000e+00,5.0,0.0,6.000000e+01,1.495800e+04,0.000000e+00,0.0,6.400000e+01,1.700000e+01,1.651800e+04,...,0.000000e+00,0.000000e+00,1.000000e+00,0.0,1.0,0.0,4.581100e+05,55.000000,17.0,64.0
50%,4.000000e+00,5.0,0.0,7.200000e+01,3.163400e+04,2.000000e+00,0.0,6.400000e+01,1.700000e+01,3.272500e+04,...,0.000000e+00,0.000000e+00,1.000000e+00,0.0,1.0,0.0,5.088930e+05,62.000000,17.0,64.0
75%,4.000000e+00,5.0,0.0,9.200000e+01,4.843800e+04,2.000000e+00,0.0,1.280000e+02,1.700000e+01,4.810900e+04,...,0.000000e+00,0.000000e+00,1.000000e+00,0.0,1.0,0.0,7.887590e+05,175.000000,17.0,64.0
max,6.000000e+00,5.0,0.0,1.499000e+03,6.553500e+04,2.000000e+00,0.0,1.280000e+02,1.700000e+01,6.553500e+04,...,1.000000e+00,1.000000e+00,6.539900e+04,0.0,1.0,0.0,1.046083e+06,427.000000,17.0,64.0


In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe() # To display in Jupyter Notebook

In [10]:
import pygwalker as pyg

walker = pyg.walk(df_all)

Box(children=(HTML(value='\n<div id="ifr-pyg-0006315256d051c3ZHrhnOwSlTvWgCIE" style="height: auto">\n    <hea…